## 5.horseIDとjockeyIDの取得

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from tqdm import tqdm
import functions
import numpy as np
import importlib

pd.options.display.max_rows = 5
pd.options.display.max_columns = 100
pd.options.display.max_colwidth = 30

In [6]:

df = pd.read_csv('2.2.add.日付_df.csv')
race_id_list = df.index.unique()
# indexをstr型に変換
print(type(race_id_list[0]))
# race_id_list = ["202101010101","202101010102"]
race_id_list = map(str,race_id_list)
# df

<class 'numpy.int64'>


In [7]:
test = {}
def scrape_race_results(race_id_lists,par_race_results={}):
  race_results = par_race_results
  for _race_id in tqdm(race_id_lists):
    # if _race_id in race_result.keys():
    #   continue
    time.sleep(1)
    try:
      # スクレイピング
      url = "https://race.netkeiba.com/race/result.html?race_id=" + _race_id
      html = requests.get(url)
      html.encoding= "EUC_JP"
      soup = BeautifulSoup(html.text, "html.parser")
      
      # horse_id
      horse_id_list = []      
      horse_a_list = soup.find('table',attrs={"summary":"全着順"}).find_all('a',attrs={"href":re.compile("/horse/")})
      for a in horse_a_list:
        horse_id = re.findall(r'\d+',a["href"])
        horse_id_list.append(horse_id[0])
      Series1 = pd.Series(data = horse_id_list,dtype=str,name="horse_id")
      # jockey_id
      jockey_id_list = []
      jockey_a_list = soup.find('table',attrs={"summary":"全着順"}).find_all('a',attrs={"href":re.compile("/jockey/")})
      for u in jockey_a_list:
        jockey_id = re.findall(r'\d+',u["href"])
        jockey_id_list.append(jockey_id[0])
      Series2 = pd.Series(data = jockey_id_list,dtype=str,name="jockey_id")
      
      results = pd.concat([Series1, Series2], axis=1)
      test[_race_id] = results
      
    except UnicodeDecodeError as e:
      # print(e)
      print(f"レースID: {_race_id} のデータはありません")
      continue
    except KeyboardInterrupt:
      break
    except TimeoutError:
      break
  return test

In [8]:
results_new = scrape_race_results(race_id_list)
# results_new

108it [02:31,  1.40s/it]


{'202101010101':       horse_id jockey_id
 0   2018105875     01181
 1   2018105258     01140
 ..         ...       ...
 14  2018104780     01144
 15  2018100737     05203
 
 [16 rows x 2 columns],
 '202101010102':       horse_id jockey_id
 0   2018105205     00666
 1   2018100974     01161
 ..         ...       ...
 12  2018105770     01172
 13  2018100706     01140
 
 [14 rows x 2 columns],
 '202101010103':       horse_id jockey_id
 0   2018100403     01157
 1   2018105166     01182
 ..         ...       ...
 10  2018102272     01015
 11  2018103483     01096
 
 [12 rows x 2 columns],
 '202101010104':       horse_id jockey_id
 0   2018100537     01182
 1   2018103489     01096
 ..         ...       ...
 13  2018101597     01176
 14  2018104222     01124
 
 [15 rows x 2 columns],
 '202101010105':       horse_id jockey_id
 0   2019104536     01157
 1   2019103120     01176
 ..         ...       ...
 11  2019106524     01182
 12  2019105105     00666
 
 [13 rows x 2 columns],
 '20210101

----

## 6.取得したデータを成形して元のデータと結合

In [21]:
for key in results_new.keys():
  results_new[key].index = [key] * len(results_new[key])
print(results_new)
results = pd.concat([results_new[key] for key in results_new.keys()],sort=False)
results

{'202101010101':                 horse_id jockey_id
202101010101  2018105875     01181
202101010101  2018105258     01140
...                  ...       ...
202101010101  2018104780     01144
202101010101  2018100737     05203

[16 rows x 2 columns], '202101010102':                 horse_id jockey_id
202101010102  2018105205     00666
202101010102  2018100974     01161
...                  ...       ...
202101010102  2018105770     01172
202101010102  2018100706     01140

[14 rows x 2 columns], '202101010103':                 horse_id jockey_id
202101010103  2018100403     01157
202101010103  2018105166     01182
...                  ...       ...
202101010103  2018102272     01015
202101010103  2018103483     01096

[12 rows x 2 columns], '202101010104':                 horse_id jockey_id
202101010104  2018100537     01182
202101010104  2018103489     01096
...                  ...       ...
202101010104  2018101597     01176
202101010104  2018104222     01124

[15 rows x 2 columns],

,horse_id,jockey_id
202101010101,2018105875,01181
202101010101,2018105258,01140
...,...,...
202102010312,2017101230,01140
202102010312,2014104973,01025


In [25]:
print(type(results))
print(len(results))
# results.index = results.index.astype(str)
print(type(results.index[0]))
print(type(df))
print(len(df))
df.index = df.index.astype(str)
print(type(df.index[0]))


<class 'pandas.core.frame.DataFrame'>
1366
<class 'str'>
<class 'pandas.core.frame.DataFrame'>
1366
<class 'str'>


In [26]:
# pd.set_option('display.max_rows',None)

df_add_id = pd.concat([df,results],axis=1)

In [27]:
df_add_id.to_csv("3.1.df_add_id.csv")

----

## 7.idのリストを作成

In [ ]:
horse_id_list = df_add_id["horse_id"].unique()
len(horse_id_list)
# horse_id_list = horse_id_list[:50]
# horse_id_list = ['2018105875',"2018102192"]

In [ ]:
# url = "https://db.netkeiba.com/horse/2019109094"
# url = "https://db.netkeiba.com/horse/2018102444"
# time.sleep(1)
# pd.read_html(url,match='日付')

In [ ]:
test = {}
def scrape_horse_results(horse_id_list,par_horse_results={}):
  horse_results = par_horse_results
  for _horse_id in tqdm(horse_id_list):
    # if _horse_id in horse_id_list.keys():
    #   continue
    time.sleep(1)
    try:
      # スクレイピング
      url = "https://db.netkeiba.com/horse/" + _horse_id
      time.sleep(1)
      # [0]をつけてリストの中だけを取得
      horse_results[_horse_id] = pd.read_html(url,match='日付',encoding='EUC-JP')[0]
    except UnicodeDecodeError as e:
      # print(e)
      # print(f"レースID: {race_id} のデータはありません")
      continue
    except KeyboardInterrupt:
      break
        
    except TimeoutError:
      break
    # print(horse_results)
      
  return horse_results

In [ ]:
horse_results = scrape_horse_results(horse_id_list)

In [ ]:
horse_results

----

## 取得したデータを成形

In [ ]:
for key in horse_results.keys():
  horse_results[key].index = [key] * len(horse_results[key])
results = pd.concat([horse_results[key] for key in horse_results.keys()],sort=False)

In [ ]:
# pd.options.display.max_rows = None

results

In [ ]:
# results.to_csv("3.2.horse_vs_data.csv")

In [ ]:
# time.sleep(2)
# df = pd.read_csv("3.2.horse_vs_data.csv")

In [ ]:
# print(df.columns)
# df = df.iloc[:,:29]
# print(df.columns)

----

## データの自動成形

In [55]:
importlib.reload(functions)

arr = [
       # 13238,13239,13334,13335,13958,13978,13993,14019,
       # 14020,14316,14317,14341,14342,14388,14548,14549,
       # 14555,14556,14718,14719,14555,14556,15056,15057,
       # 15129,14555,14556,15057,15129,15130,15316,15317,
       # 15378,15379,15425,15426,15466,15517,15518,15519,
       # 15557,15558,15638,15639,15665,15666,15799,15800,
       # 15838,15839,15859,15860
       ]

clean_index = list(map(lambda x: x-1,arr))
# l,l_length,columns_len = Molding(clean_index,"3.2.horse_vs_data.csv")

l,l_length,columns_len,おかしいリスト = functions.Molding_new(clean_index,"3.4.horse_vs_data.csv")
おかしいリスト

[]

In [ ]:
print(f"全体のlen{l_length}")
print(f"カラムのlen{columns_len}")

In [ ]:
for i in l :
  if len(i) == 32:pass
  else: print(len(i), i)

In [ ]:

add = np.array(l[1:]).reshape(l_length-1,32)


10秒

In [ ]:
df_add = pd.DataFrame(add, columns=l[0])
# print(df_add)

In [ ]:
df_add.to_csv('3.4.horse_vs_data.csv',index=False)

In [7]:
df = pd.read_csv("3.4.horse_vs_data.csv")

In [8]:
pd.options.display.max_rows = None
for i in df.columns:
  print(df[i].value_counts())

2021/06/13    183
2021/06/12    166
2021/07/10    163
2021/08/21    160
2021/06/19    158
2021/08/14    157
2021/07/03    155
2021/07/04    149
2021/08/15    148
2021/06/26    110
2021/06/27    105
2021/06/20    103
2021/07/24     99
2021/08/29     96
2021/09/04     93
2021/09/05     92
2021/07/18     90
2021/07/17     88
2021/07/25     84
2021/08/28     81
2021/08/08     79
2021/07/11     75
2021/08/01     75
2021/07/31     75
2021/03/07     74
2021/04/25     74
2021/01/17     73
2021/03/06     73
2021/02/13     73
2021/04/10     73
2021/04/17     72
2020/12/06     71
2020/11/14     68
2021/02/20     65
2021/02/06     65
2021/05/15     65
2021/05/23     64
2021/02/28     64
2021/01/24     64
2021/01/23     63
2021/03/28     62
2021/08/07     62
2021/03/27     62
2021/02/07     61
2021/02/27     61
2021/12/12     60
2021/01/16     60
2020/09/05     59
2021/11/21     59
2021/11/07     59
2021/01/31     59
2022/01/23     58
2021/08/22     58
2021/02/14     58
2020/11/07     57
2020/10/10

In [9]:
# df